In [1]:
using PowerModels, JuMP, Ipopt

In [2]:
data = parse_matpower("case5.m");

[warn | PowerModels]: this code only supports angmin values in -90 deg. to 90 deg., tightening the value on branch 4 from -360.0 to -60.0 deg.
[warn | PowerModels]: this code only supports angmax values in -90 deg. to 90 deg., tightening the value on branch 4 from 360.0 to 60.0 deg.
[warn | PowerModels]: this code only supports angmin values in -90 deg. to 90 deg., tightening the value on branch 1 from -360.0 to -60.0 deg.
[warn | PowerModels]: this code only supports angmax values in -90 deg. to 90 deg., tightening the value on branch 1 from 360.0 to 60.0 deg.
[warn | PowerModels]: this code only supports angmin values in -90 deg. to 90 deg., tightening the value on branch 5 from -360.0 to -60.0 deg.
[warn | PowerModels]: this code only supports angmax values in -90 deg. to 90 deg., tightening the value on branch 5 from 360.0 to 60.0 deg.
[warn | PowerModels]: this code only supports angmin values in -90 deg. to 90 deg., tightening the value on branch 2 from -360.0 to -60.0 deg.
[warn

In [3]:
N = sort!(collect(keys(data["bus"])))
num_nodes = length(N)
Vmin = [data["bus"][i]["vmin"] for i in N]
Vmax = [data["bus"][i]["vmax"] for i in N]
L = sort!(collect(keys(data["branch"])))
lines = [(data["branch"][k]["f_bus"], data["branch"][k]["t_bus"]) for k in L]
r = [data["branch"][i]["br_r"] for i in L]
x = [data["branch"][i]["br_x"] for i in L]
b = [data["branch"][i]["b_fr"] for i in L]
τ = [data["branch"][i]["tap"] for i in L]
ψ = [data["branch"][i]["shift"] for i in L]
smax = [ "rate_a" in keys(data["branch"][i]) ? data["branch"][i]["rate_a"] : 0 for i in L]
z = [r[j] + im * x[j] for j in eachindex(r)]
num_loads = length(data["load"])
Pd = zeros(num_nodes)
Qd = zeros(num_nodes)
for i in values(data["load"])
    node = i["load_bus"]
    Pd[node] = i["pd"]
    Qd[node] = i["qd"]
end
num_gens = length(data["gen"])
Gnode = [ i["gen_bus"] for i in values(data["gen"]) ]
Pmax = [ i["pmax"] for i in values(data["gen"]) ]
Pmin = [ i["pmin"] for i in values(data["gen"]) ]
Qmax = [ i["qmax"] for i in values(data["gen"]) ]
Qmin = [ i["qmin"] for i in values(data["gen"]) ]
c = [ i["cost"][1] for i in values(data["gen"]) ]

#Pd = Dict(i => data["loads"][i]["pd"] for i in keys(data["loads"]))
#Pnode = Dict(i => data["loads"][i]["pd"] for i in keys(data["loads"]))


Yff = [(z[j]^(-1) + im * b[j] / 2)/τ[j]^2 for j in eachindex(z)]
Yft = [-z[j]^(-1) / (τ[j] * exp(-im*ψ[j])) for j in eachindex(z)]
Ytf = [-z[j]^(-1) / (τ[j] * exp(im*ψ[j])) for j in eachindex(z)]
Ytt = [z[j]^(-1) + im * b[j] / 2 for j in eachindex(z)];


In [4]:
Vm = Dict()
for i in eachindex(N)
    Vm[i] = zeros(2*num_nodes, 2*num_nodes)
    Vm[i][i,i] = 1
    Vm[i][i+num_nodes,i+num_nodes] = 1
end

In [5]:
Pf = Dict()
Qf = Dict()
Pt = Dict()
Qt = Dict()
for k in eachindex(L)
    (i,j) = lines[k]
    Pf[(i,j)] = zeros(2*num_nodes, 2*num_nodes)
    Qf[(i,j)] = zeros(2*num_nodes, 2*num_nodes)
    Pt[(i,j)] = zeros(2*num_nodes, 2*num_nodes)
    Qt[(i,j)] = zeros(2*num_nodes, 2*num_nodes)

    Pf[(i,j)][i,i] = Yff[k].re
    Pf[(i,j)][i+num_nodes, i+num_nodes] = Yff[k].re
    Pf[(i,j)][i,j] = 1/2 * Yft[k].re
    Pf[(i,j)][j,i] = 1/2 * Yft[k].re
    Pf[(i,j)][i+num_nodes,j+num_nodes] = 1/2 * Yft[k].re
    Pf[(i,j)][j+num_nodes,i+num_nodes] = 1/2 * Yft[k].re
    Pf[(i,j)][i+num_nodes,j] = 1/2 * Yft[k].im
    Pf[(i,j)][j,i+num_nodes] = 1/2 * Yft[k].im
    Pf[(i,j)][i,j+num_nodes] = -1/2 * Yft[k].im
    Pf[(i,j)][j+num_nodes,i] = -1/2 * Yft[k].im

    Qf[(i,j)][i,i] = -Yff[k].im
    Qf[(i,j)][i+num_nodes, i+num_nodes] = -Yff[k].im
    Qf[(i,j)][i,j] = -1/2 * Yft[k].im
    Qf[(i,j)][j,i] = -1/2 * Yft[k].im
    Qf[(i,j)][i+num_nodes,j+num_nodes] = -1/2 * Yft[k].im
    Qf[(i,j)][j+num_nodes,i+num_nodes] = -1/2 * Yft[k].im
    Qf[(i,j)][i+num_nodes,j] = 1/2 * Yft[k].re
    Qf[(i,j)][j,i+num_nodes] = 1/2 * Yft[k].re
    Qf[(i,j)][i,j+num_nodes] = -1/2 * Yft[k].re
    Qf[(i,j)][j+num_nodes,i] = -1/2 * Yft[k].re

    Pt[(i,j)][j,j] = Ytt[k].re
    Pt[(i,j)][j+num_nodes, j+num_nodes] = Ytt[k].re
    Pt[(i,j)][i,j] = 1/2 * Ytf[k].re
    Pt[(i,j)][j,i] = 1/2 * Ytf[k].re
    Pt[(i,j)][i+num_nodes,j+num_nodes] = 1/2 * Ytf[k].re
    Pt[(i,j)][j+num_nodes,i+num_nodes] = 1/2 * Ytf[k].re
    Pt[(i,j)][i,j+num_nodes] = 1/2 * Ytf[k].im
    Pt[(i,j)][j+num_nodes,i] = 1/2 * Ytf[k].im
    Pt[(i,j)][i+num_nodes,j] = -1/2 * Ytf[k].im
    Pt[(i,j)][j,i+num_nodes] = -1/2 * Ytf[k].im

    Qt[(i,j)][j,j] = -Ytt[k].im
    Qt[(i,j)][j+num_nodes, j+num_nodes] = -Ytt[k].im
    Qt[(i,j)][i,j] = -1/2 * Ytf[k].im
    Qt[(i,j)][j,i] = -1/2 * Ytf[k].im
    Qt[(i,j)][i+num_nodes,j+num_nodes] = -1/2 * Ytf[k].im
    Qt[(i,j)][j+num_nodes,i+num_nodes] = -1/2 * Ytf[k].im
    Qt[(i,j)][i,j+num_nodes] = 1/2 * Ytf[k].re
    Qt[(i,j)][j+num_nodes,i] = 1/2 * Ytf[k].re
    Qt[(i,j)][i+num_nodes,j] = -1/2 * Ytf[k].re
    Qt[(i,j)][j,i+num_nodes] = -1/2 * Ytf[k].re;
end

In [20]:
num_lines = length(L)
m = Model(Ipopt.Optimizer);
@variable(m, v[1:2*num_nodes], start = .1)
@variable(m, W[1:2*num_nodes,1:2*num_nodes], start = .1)
@variable(m, pf[1:num_lines], start = .1)
@variable(m, pt[1:num_lines], start = .1)
@variable(m, qf[1:num_lines], start = .1)
@variable(m, qt[1:num_lines], start = .1)
@variable(m, pG[1:num_gens], start = .1)
@variable(m, qG[1:num_gens], start = .1)


# 1b, 1c: define pf, pt, qf, qt
for i in eachindex(L)
    @constraint(m, pf[i] == sum(Pf[lines[i]] .* W))
    @constraint(m, qf[i] == sum(Pf[lines[i]] .* W))
    @constraint(m, pt[i] == sum(Pf[lines[i]] .* W))
    @constraint(m, qt[i] == sum(Pf[lines[i]] .* W))
end

#1d, 1e: active and reactive power balance
for i in eachindex(N)
    if any(Gnode .== i)
        @constraint(m, sum(VariableRef[pf[j] for j in eachindex(L) if lines[j][1] == i]) 
                      +sum(VariableRef[pt[j] for j in eachindex(L) if lines[j][2] == i])
                      -sum(VariableRef[pG[j] for j in eachindex(Gnode) if Gnode[j] == i]) + Pd[i] == 0)
        @constraint(m, sum(VariableRef[qf[j] for j in eachindex(L) if lines[j][1] == i])
                      +sum(VariableRef[qt[j] for j in eachindex(L) if lines[j][2] == i]) 
                      -sum(VariableRef[qG[j] for j in eachindex(Gnode) if Gnode[j] == i]) + Qd[i] == 0)
    else
        @constraint(m, sum(VariableRef[pf[j] for j in eachindex(L) if lines[j][1] == i]) + 
                       sum(VariableRef[pt[j] for j in eachindex(L) if lines[j][2] == i]) + Pd[i] == 0)
        @constraint(m, sum(VariableRef[qf[j] for j in eachindex(L) if lines[j][1] == i]) + 
                       sum(VariableRef[qt[j] for j in eachindex(L) if lines[j][2] == i]) + Qd[i] == 0)
    end
end

# 1f: power capacity
for i in 1:num_gens
    @constraint(m, Pmin[i] <= pG[i] <= Pmax[i])
    @constraint(m, Qmin[i] <= qG[i] <= Qmax[i])
end

# 1g: voltage capacity
for i in 1:num_nodes
    @constraint(m, Vmin[i]^2 <= sum(Vm[i] .* W) <= Vmax[i]^2)
end

# 1h: thermal line capacity
for i in eachindex(L)
    if smax[i] != 0
        @constraint(m, pf[i]^2 + qf[i]^2 <= smax[i]^2)
    end
end

# 1i: W rank-1, symmetric PSD
for i in 1:2*num_nodes, j in 1:2*num_nodes
    @constraint(m, W[i,j] == v[i] * v[j])
end

@objective(m, Min, sum(c .* pG));

In [16]:
all_variables(m)

144-element Array{VariableRef,1}:
 v[1]  
 v[2]  
 v[3]  
 v[4]  
 v[5]  
 v[6]  
 v[7]  
 v[8]  
 v[9]  
 v[10] 
 W[1,1]
 W[2,1]
 W[3,1]
 ⋮     
 qt[5] 
 qt[6] 
 pG[1] 
 pG[2] 
 pG[3] 
 pG[4] 
 pG[5] 
 qG[1] 
 qG[2] 
 qG[3] 
 qG[4] 
 qG[5] 

In [21]:
optimize!(m)
println(objective_value(m))

This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      588
Number of nonzeros in inequality constraint Jacobian.:       44
Number of nonzeros in Lagrangian Hessian.............:      104

Total number of variables............................:      144
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:      134
Total number of inequality constraints...............:       32
        inequality constraints with only lower bounds:       15
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:       17

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 

In [33]:
data["branch"]["1"]

Dict{String,Any} with 19 entries:
  "br_r"        => 0.00281
  "rate_a"      => 4.0
  "shift"       => 0.0
  "rate_b"      => 4.0
  "br_x"        => 0.0281
  "rate_c"      => 4.0
  "g_to"        => 0.0
  "g_fr"        => 0.0
  "source_id"   => Any["branch", 1]
  "b_fr"        => 0.00356
  "f_bus"       => 1
  "br_status"   => 1
  "t_bus"       => 2
  "b_to"        => 0.00356
  "index"       => 1
  "angmin"      => -1.0472
  "angmax"      => 1.0472
  "transformer" => false
  "tap"         => 1.0